In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
#Load csv Form Part 1
file = '../WeatherPy/output/weather_df.csv'
weather_df = pd.read_csv(file)
weather_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Barrow,71.2906,-156.7887,19.40,100,90,9.22,US,1619556025
1,Mitsamiouli,-11.3847,43.2844,81.03,100,21,8.37,KM,1619556106
2,Westport,41.1415,-73.3579,63.00,100,1,12.66,US,1619556106
3,Iqaluit,63.7506,-68.5145,24.80,100,20,3.44,CA,1619556107
4,Alice Springs,-23.7000,133.8833,44.60,100,0,5.75,AU,1619556107


# Humidity Heatmap

-Configure gmaps.

-Use the Lat and Lng as locations and Humidity as the weight.

-Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=np.max(humidity), 
                                 point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.

In [5]:
weather_ideal = weather_df.loc[(weather_df["Max Temperature (F)"] <= 80) &
                                           (weather_df["Max Temperature (F)"] > 70) &
                                           (weather_df["Wind Speed"] <= 10) &
                                           (weather_df["Cloudiness"] == 0 )]
weather_ideal

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
55,Baikunthpur,23.2500,82.5500,74.68,94,0,3.27,IN,1619556128
66,Yangi Marg`ilon,40.4272,71.7189,71.60,94,0,4.61,UZ,1619556132
194,Mayor Pablo Lagerenza,-19.9309,-60.7718,76.24,83,0,5.48,PY,1619556188
229,Tikrit,34.6071,43.6782,75.20,81,0,6.53,IQ,1619556200
240,Saint-Philippe,-21.3585,55.7679,75.20,80,0,3.44,RE,1619556209
250,Baghdad,33.3406,44.4009,71.60,78,0,5.75,IQ,1619555915
324,Santa Cruz de la Sierra,-17.8000,-63.1667,74.25,70,0,4.85,BO,1619556243
345,Saint-Pierre,-21.3393,55.4781,75.20,68,0,3.44,RE,1619556256
385,Rivadavia,-33.1905,-68.4608,75.20,61,0,8.05,AR,1619556052


# Hotel Map

In [6]:
hotel_df = weather_ideal

hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-6-59ffa3c4073b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
55,Baikunthpur,23.2500,82.5500,74.68,94,0,3.27,IN,1619556128,
66,Yangi Marg`ilon,40.4272,71.7189,71.60,94,0,4.61,UZ,1619556132,
194,Mayor Pablo Lagerenza,-19.9309,-60.7718,76.24,83,0,5.48,PY,1619556188,
229,Tikrit,34.6071,43.6782,75.20,81,0,6.53,IQ,1619556200,
240,Saint-Philippe,-21.3585,55.7679,75.20,80,0,3.44,RE,1619556209,
250,Baghdad,33.3406,44.4009,71.60,78,0,5.75,IQ,1619555915,
324,Santa Cruz de la Sierra,-17.8000,-63.1667,74.25,70,0,4.85,BO,1619556243,
345,Saint-Pierre,-21.3393,55.4781,75.20,68,0,3.44,RE,1619556256,
385,Rivadavia,-33.1905,-68.4608,75.20,61,0,8.05,AR,1619556052,


In [7]:
#Set parameters to search for hotels with 500 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#get coodinates from DF:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
      
    
    params["location"] = f"{lat},{lng}"
    
    # API request:
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Could not find result")
        
        print("-------------------------")

Retrieving Results for Index 55: Baikunthpur.
Closest hotel in Baikunthpur is Hotel Ganga Shree.
Retrieving Results for Index 66: Yangi Marg`ilon.


C:\Users\Carlos A\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest hotel in Yangi Marg`ilon is Diyor hotel.
Retrieving Results for Index 194: Mayor Pablo Lagerenza.
Could not find result
-------------------------
Retrieving Results for Index 229: Tikrit.
Closest hotel in Tikrit is فندق تكريت بلازا.
Retrieving Results for Index 240: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
Retrieving Results for Index 250: Baghdad.
Closest hotel in Baghdad is Baghdad Hotel.
Retrieving Results for Index 324: Santa Cruz de la Sierra.
Closest hotel in Santa Cruz de la Sierra is Los Tajibos Hotel & Convention Center.
Retrieving Results for Index 345: Saint-Pierre.
Closest hotel in Saint-Pierre is Villa Delisle, Hotel & Spa.
Retrieving Results for Index 385: Rivadavia.
Closest hotel in Rivadavia is Hotel Pasambay.


In [8]:
hotel_df

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
55,Baikunthpur,23.2500,82.5500,74.68,94,0,3.27,IN,1619556128,Hotel Ganga Shree
66,Yangi Marg`ilon,40.4272,71.7189,71.60,94,0,4.61,UZ,1619556132,Diyor hotel
194,Mayor Pablo Lagerenza,-19.9309,-60.7718,76.24,83,0,5.48,PY,1619556188,
229,Tikrit,34.6071,43.6782,75.20,81,0,6.53,IQ,1619556200,فندق تكريت بلازا
240,Saint-Philippe,-21.3585,55.7679,75.20,80,0,3.44,RE,1619556209,Les Embruns Du Baril
250,Baghdad,33.3406,44.4009,71.60,78,0,5.75,IQ,1619555915,Baghdad Hotel
324,Santa Cruz de la Sierra,-17.8000,-63.1667,74.25,70,0,4.85,BO,1619556243,Los Tajibos Hotel & Convention Center
345,Saint-Pierre,-21.3393,55.4781,75.20,68,0,3.44,RE,1619556256,"Villa Delisle, Hotel & Spa"
385,Rivadavia,-33.1905,-68.4608,75.20,61,0,8.05,AR,1619556052,Hotel Pasambay


In [9]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))